# Front matter

## Welcome

My research interests:

* High-dimensional statistics (more variables than observations)
* Predictive modeling and methodology for validation
* Metagenomic profiling of the human microbiome
* Multi-omic data analysis for cancer
* http://www.waldronlab.io

## The textbook

* [Biomedical Data Science](book) by Irizarry and Love ([ePub version] on Leanpub)
* [Source repository](https://github.com/genomicsclass/labs)

![Biomedical Data Science book cover](https://d2sofvawe08yqg.cloudfront.net/dataanalysisforthelifesciences/hero2x?1549465432)

## Day 1 outline

- RStudio vs Jupyter Notebooks
- Random variables and distributions
- Hypothesis testing for one or two samples (t-test, Wilcoxon test, etc)
- Hypothesis testing for categorical variables (Fisher's Test, Chi-square test)
- Some essential R classes and related Bioconductor classes
- Introduction to `dplyr`
    
- [Book](book) chapters 0 and 1

## Learning objectives

- Perform basic data manipulation/exploration in R and dplyr
- Identify key R and Bioconductor data classes
- Define random variables and distinguish them from non-random ones
- Distinguish population from sampling distributions
- Interpret t-tests and confidence intervals
- Identify when to use Fisher's Exact Test and Chi-square Test

In [ ]:
installOrLoad <- function(pkgs){
    for (pkg in pkgs){
        if(!require(pkg, character.only=TRUE)){
            BiocManager::install(pkg)
            require(pkg, character.only=TRUE)
        }
    }
}
installOrLoad(c("S4Vectors", "IRanges", "nycflights13", "dplyr", "TeachingDemos", "epitools"))

# Random Variables and Distributions

## Random Variables

- **A random variable**: any characteristic that can be measured or categorized, and where any particular outcome is determined at least partially by chance.

- Examples:
    - number of new diabetes cases in population in a given year
    - The weight of a randomly selected individual in a population

- Types:
    - Categorical random variable (e.g. disease / healthy)
    - Discrete random variable (e.g. sequence read counts)
    - Continuous random variable (e.g. normalized qPCR intensity)

## Random Variables - examples

Normally distributed random variable with mean $\mu = 0$ / standard deviation $\sigma = 1$, and a sample of $n=100$

In [ ]:
x = rnorm(100)
res = hist(x, main = "Standard Normal Distribution\n mean 0, std. dev. 1", prob =
               TRUE)
xdens = seq(min(res$breaks), max(res$breaks), by = 0.01)
lines(xdens, dnorm(xdens))

## Random Variables - examples

Poisson distributed random variable ($\lambda = 2$), and a sample of $n=100$.

In [ ]:
x = rpois(100, lambda = 2)
res = hist(
    x,
    main = "Poisson Distribution",
    prob = FALSE,
    col = "lightgrey",
    breaks = seq(-0.5, round(max(x)) + 0.5, by = 0.5)
)
xdens = seq(min(x), max(x), by = 1)
lines(xdens, length(x) * dpois(xdens, lambda = 2), lw = 2)

## Random Variables - examples

Negative Binomially distributed random variable ($size=30, \mu=2$), and a sample of $n=100$.

In [ ]:
x = rnbinom(100, size = 30, mu = 2)
res = hist(
    x,
    main = "Negative Binomial Distribution",
    prob = FALSE,
    col = "lightgrey",
    breaks = seq(-0.5, round(max(x)) + 0.5, by = 0.5)
)
xdens = seq(min(x), max(x), by = 1)
lines(xdens, length(x) * dnbinom(xdens, size = 30, mu = 2), lw = 2)

## Random Variables - examples

- Binomial Distribution random variable ($size=20, prob=0.25$), and a sample of $n=100$.
    - use for binary outcomes

In [ ]:
x = rbinom(100, size = 20, prob = 0.25)
res = hist(
    x,
    main = "Binomial Distribution",
    prob = FALSE,
    col = "lightgrey",
    breaks = seq(-0.5, round(max(x)) + 0.5, by = 0.5)
)
xdens = seq(min(x), max(x), by = 1)
lines(xdens, length(x) * dbinom(xdens, size = 20, prob = 0.25), lw = 2)

# Hypothesis Testing

## Why hypothesis testing?

- Allows yes/no statements, e.g. whether:
    - a population mean has a hypothesized value, or
    - an intervention causes a measurable effect relative to a control group

- Example questions with yes/no answers:
    - Is a gene differentially expressed between two populations?
    - Do hypertensive, smoking men have the same mean cholesterol level as the general population?

- Hypothesis testing is not the only framework for inferential statistics, e.g.:
    - confidence intervals
    - posterior probabilities (Bayesian statistics)
    - read [p-values are just the tip of the iceberg](http://www.nature.com/news/statistics-p-values-are-just-the-tip-of-the-iceberg-1.17412)

## Logic of hypothesis testing

* Hypotheses are made about *populations* based on inference from *samples*
* We make inference from *samples* because we usually cannot observe the entire population

<center>
![Sample vs Population](https://keydifferences.com/wp-content/uploads/2016/06/population-vs-sample2.jpg)
</center>

## One and two-sample hypothesis tests of mean

**One-sample hypothesis test of mean** - sample comes from one of two population distributions:

1. usual distribution that has been true in the past
    - $H_0: \mu = \mu_0$ (null hypothesis)
2. a potentially new distribution induced by an intervention or changing condition
    - $H_A: \mu \neq \mu_0$ (alternative hypothesis)

**Two-sample hypothesis test of means** - two samples are drawn, either:

1. from a single population
    - $H_0: \mu_1 = \mu_2$ (null hypothesis)
2. from two different populations
    - $H_A: \mu_1 \neq \mu_2$ (alternative hypothesis)

## Population vs sampling distributions

- Population distributions
    + Each realization / point is an individual
    
- Sampling distributions
    + Each realization / point is a sample
    + distribution depends on sample size
    + large sample distributions are given by the Central Limit Theorem

- **Hypothesis testing is about sampling distributions**
    + Did my sample likely come from that distribution?

## Note about the Central Limit Theorem

When sample size is large, the average $\bar{Y}$ of a random sample:
 1. Follows a normal distribution
 2. The normal distribution has mean entered at the population average $\mu_Y$ 
 3. with standard deviation equal to the population standard deviation $\sigma_Y$, divided by the square root of the sample size $N$. We refer to the standard deviation of the distribution of a random variable as the random variable's _standard error_.

Thanks to CLT, linear modeling, t-tests, ANOVA are all guaranteed to work reasonably well for large samples (more than ~30 observations).

Go to online demo: http://onlinestatbook.com/stat_sim/sampling_dist/

## Example applications of hypothesis tests for continuous variables

1. Do study participants on a diet lose weight compared to a control group?
2. Does a gene knockout result in less viable cell colonies, as measured by the number of living cells in replicate experiments?
3. Is *Prevotella copri* more abundant in the guts of vegans than of meat-eaters?
4. Do infants born in this region have a higher birthweight than the national average?

These are research hypotheses. What are the corresponding *null hypotheses*?

## The z-tests

- "$z$" refers to the **Standard Normal Distribution**: $N(\mu=0, \sigma=1)$
- In a one-sample test, only a single sample is drawn:
    - $H_0: \mu = \mu_0$
- In a two-sample test, two samples are drawn *independently*:
    - $H_0: \mu_1 = \mu_2$
- A paired test is one sample of paired measurements, e.g.:
    - pain level in individuals before and after treatment
       + $H_0: \mu_{before} = \mu_{after}$
    - microbial abundance in upper and lower intestines in same individuals
       + $H_0: \mu_{upper} = \mu_{lower}$

## When to use z-tests

* $\frac{{}\bar{x} - \mu}{\sigma}$ and $\frac{\bar{x_1} - \bar{x_2}}{\sigma}$
are z-distributed *if*:
    - **the standard deviation $\sigma$ for the population is known in advance**
    - the population values are normally distributed
    - the population values are slightly skewed but n > 15
    - the population values are quite skewed but n > 30

## Example of one-sample z-test

In [ ]:
library(TeachingDemos)
set.seed(1)
(weights = rnorm(10, mean = 75, sd = 10))
TeachingDemos::z.test(
    x = weights,
    mu = 70,  #specified for population under H0 because this is a one-sample test
    stdev = 10, #specified for population because this is a z-test
    alternative = "two.sided"
)

## The t-tests

- The t-tests are for small samples and do _not_ rely on the Central Limit Theorem
- In a one-sample test, only a single sample is drawn:
    - $H_0: \mu = \mu_0$
- In a two-sample test, two samples are drawn *independently*:
    - $H_0: \mu_1 = \mu_2$
- A paired test is one sample of paired measurements, e.g.:
    - individuals before and after treatment

## When to use t-tests

* $\frac{{}\bar{x} - \mu}{s}$ and $\frac{\bar{x_1} - \bar{x_2}}{s}$
are t-distributed *if*:
    - **the standard deviation $s$ is estimated from the sample**
    - the population values are normally distributed
    - the population values are slightly skewed but n > 15
    - the population values are quite skewed but n > 30

* Wilcoxon tests are an alternative for non-normal populations
    - e.g. rank data
    - data where ranks are more informative than means
    - *not* when many ranks are arbitrary

## Example of one-sample t-test

Note that here we do not specify the standard deviation, because it is estimated from the sample.

In [ ]:
stats::t.test(
    x = weights,
    mu = 70,  #specified for population under H0 because this is a one-sample test
    alternative = "two.sided"
)

In this particular example the p-value is smaller than from the *z-test*, but in general it would be **less powerful** than a *z-test* if its assumptions are met.

# Confidence intervals

## Why confidence intervals?

- The above procedures produced both p-values and confidence intervals
- p-values are useful only for deciding whether to reject $H_0$
- p-values do not report effect size (observed difference).
- p-values only indicate statistical significance which does not guarantee scientific/clinical significance.

Confidence intervals provide a **probable range for the true population mean**.

In [ ]:
set.seed(1)
reps <- 100
N <- 30
alpha <- 0.05
my.conf.ints <- replicate(reps, t.test(rnorm(N), conf.level = 1-alpha)$conf.int)
plot(x = rep(range(my.conf.ints), reps / 2),
     y = 1:reps,
     main = paste(reps, "samples of size N=", N),
     type = "n")
abline(v = 0, lw = 2, lty = 3)
for (i in seq(1, reps)) {
    acceptH0 <- my.conf.ints[1, i] < 0 & my.conf.ints[2, i] > 0
    segments(
        x0 = my.conf.ints[1, i],
        y0 = i,
        x1 = my.conf.ints[2, i],
        y1 = i,
        lw = ifelse(acceptH0, 1, 2),
        col = ifelse(acceptH0, "black", "red")
    )
}

## Intervals for any confidence level

- If the sampling distribution is normal with standard error $SE$, then a confidence interval for the population mean is:
    - $\bar{X} \pm 1.96 \times SE$ (95% CI, normal sampling distribution)
    - $\bar{X} \pm z_{\alpha / 2}^{crit} \times SE$ (normal sampling distribution)
    - $\bar{X} \pm t_{\alpha / 2, df}^{crit} \times SE$ (t sampling distribution)
- The part after the $\pm$ is called the _margin of error_

## Confidence intervals vs. hypothesis testing

- Confidence intervals can be used for hypothesis testing
    - reject $H_0$ if the "null value" is not contained in the CI
- Do **not** use overlap between two CIs for hypothesis test
    - for a two sample hypothesis test $H_0: \mu_1 = \mu_2$, must construct a single confidence interval for $\mu_1 - \mu_2$

## Q & A: confidence intervals

Which of the following are true?

- The 95% CI contains 95% of the values in the population
- The 95% CI will contain the population mean, 19 times out of 20
- The 95% CI is 95% probable to contain the population mean
- The 99% CI will be wider than the 95% CI


# Hypothesis testing for categorical variables

## Lady Tasting Tea

- The Lady in question claimed to be able to tell whether the tea or the milk was added first to a cup
- Fisher proposed to give her eight cups, four of each variety, in random order
    - the Lady is **fully informed** of the experimental method
    - $H_0$: the Lady has no ability to tell which was added first

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Nice_Cup_of_Tea.jpg/330px-Nice_Cup_of_Tea.jpg" alt="Lady tasting tea" align="middle" style="height: 200px;">
</center>

Source: https://en.wikipedia.org/wiki/Lady_tasting_tea

## Fisher's Exact Test

p-value is the probability of the observed number of successes, or more, under $H_0$

<table>
<caption>Tea-Tasting Distribution</caption>
<tr>
<th scope="col">Success count</th>
<th scope="col">Permutations of selection</th>
<th scope="col">Number of permutations</th>
</tr>
<tr>
<td>0</td>
<td>oooo</td>
<td>1 × 1 = 1</td>
</tr>
<tr>
<td>1</td>
<td>ooox, ooxo, oxoo, xooo</td>
<td>4 × 4 = 16</td>
</tr>
<tr>
<td>2</td>
<td>ooxx, oxox, oxxo, xoxo, xxoo, xoox</td>
<td>6 × 6 = 36</td>
</tr>
<tr>
<td>3</td>
<td>oxxx, xoxx, xxox, xxxo</td>
<td>4 × 4 = 16</td>
</tr>
<tr>
<td>4</td>
<td>xxxx</td>
<td>1 × 1 = 1</td>
</tr>
<tr>
<th colspan="2" scope="row">Total</th>
<td>70</td>
</tr>
</table>

What do you notice about all these combinations?

## Notes on Fisher's Exact Test

- Can also be applied to rxc tables
- Remember that the margins of the table are *fixed by design*
- Also referred to as the Hypergeometric Test
- Exact p-values are difficult (and unnecessary) for large samples
    - `fisher.test(x, y = NULL, etc, simulate.p.value = FALSE)`

## Notes on Fisher's Exact Test (cont'd)

- Has been applied (**with peril!**) to gene set analysis, e.g.:
    - 10 of my top 100 genes are annotated with the cytokinesis GO term
    - 465 of 21,000 human genes are annotated with the cytokinesis GO term
    - Are my top 100 genes enriched for cytokinesis process?
- Problems with this analysis:
    - Main problem: top-n genes tend to be correlated, so their selections are not independent trials
    - Secondary: does not match design for $H_0$
- Alternative: permutation test repeating all steps

## Chi-squared test

- Test of independence for rxc table (two categorical variables)
- Does not assume the margins are fixed by design
    - i.e., the number of cups of tea with milk poured first can be random, and the Lady doesn't know how many
    - more common in practice
    - classic genomics example is GWAS
- $H_0$: the two variables are independent
- $H_A$: there is an association between the variables

## Application to GWAS

* Interested in association between disease and some potential causative factor
* In a case-control study, the numbers of cases and controls are fixed, but the other variable is not
* In a prospective or longitudinal cohort study, neither the number of cases or the other variable are fixed

In [ ]:
disease = factor(c(rep(0, 180), rep(1, 20), rep(0, 40), rep(1, 10)),
                 labels = c("control", "cases"))
genotype = factor(c(rep("AA/Aa", 204), rep("aa", 46)),
                  levels = c("AA/Aa", "aa"))
set.seed(1)
genotype <- sample(genotype)
dat <- data.frame(disease, genotype)
summary(dat)

## Application to GWAS (cont'd)

In [ ]:
table(disease, genotype)
chisq.test(disease, genotype)
chisq.test(disease, genotype, simulate.p.value = TRUE)

## Application to GWAS (cont'd)

Note that the result says nothing about *how* the departure from independence occurs

In [ ]:
library(epitools)
epitools::oddsratio(genotype, disease, method = "wald")$measure

(the default is whichever level is first alphabetically!)

## Summary - two categorical variables

- Choice between Fisher's Exact Test and Chi-square test is determined by experimental design
- If any counts in the table are less than 5, can use `simulate.p.value=TRUE` to get a more accurate p-value from the chi-square test
- Both assume independent observations (important!!)
- In a GWAS, correction for multiple testing is required
- Can also use logistic regression for two categorical variables

# Summary - hypothesis testing

## Power and type I and II error

| True state of nature | Result of test             |                                                 |
|----------------------|----------------------------|-------------------------------------------------|
|                      | **Reject $H_0$**                                      | **Fail to reject $H_0$**                                 |
| $H_0$ TRUE           | Type I error, probability = $\alpha$ | No error, probability = $1-\alpha$                     |
| $H_0$ FALSE          | No error, probability is called power = $1-\beta$    | Type II error, probability = $\beta$ (false negative) |

## Use and mis-use of the p-value

- The p-value is the probability of observing a sample statistic _as or more extreme_ as you did, _assuming that $H_0$ is true_
- The p-value is a **random variable**:
    - **don't** treat it as precise.
    - **don't** do silly things like try to interpret differences or ratios between p-values
    - **don't** lose track of test assumptions such as independence of observations
    - **do** use a moderate p-value cutoff, then use some effect size measure for ranking
- Small p-values are particularly:
    - variable under repeated sampling, and
    - sensitive to test assumptions

## Use and mis-use of the p-value (cont'd)

- If we fail to reject $H_0$, is the probability that $H_0$ is true equal to ($1-\alpha$)?  (Hint: NO NO NO!)
    - Failing to reject $H_0$ does not mean $H_0$ is true
    - "No evidence of difference $\neq$ evidence of no difference"
- Statistical significance vs. practical significance
    - As sample size increases, point estimates such as the mean are more precise
    - With large sample size, small differences in means may be statistically significant but not practically significant
- Although $\alpha = 0.05$ is a common cut-off for the p-value, there is no set border between "significant" and "insignificant," only increasingly strong evidence against $H_0$ (in favor of $H_A$) as the p-value gets smaller.

# R - basic usage

## Tips for learning R

Pseudo code                                   |   Example code
--------------------------------------------  |   -------------------
library(packagename)                          | library(dplyr)
?functionname                                 | ?select
?package::functionname                        | ?dplyr::select
? 'Reserved keyword or symbol' \color{blue}{(or backticks)} | ? '%>%'
??searchforpossiblyexistingfunctionandortopic | ??simulate
help(package = "loadedpackage")               | help("dplyr")
browseVignettes("packagename")                | browseVignettes("dplyr")

**Table credit: Marcel Ramos**

## Installing Packages the Bioconductor Way

- See the [Bioconductor](http://www.bioconductor.org/install) site for more info

Pseudo code:
```
install.packages("BiocManager") #from CRAN
packages <- c("packagename", "githubuser/repository", "biopackage")
BiocManager::install(packages)
BiocManager::valid()  #check validity of installed packages
```
- Works for CRAN, GitHub, and Bioconductor packages!


## Note about installing `devtools`

- Useful for building packages
- Download and install from GitHub, directly or via `BiocManager::install()`
- Installation dependent on OS ([Rtools](https://cran.r-project.org/bin/windows/Rtools/) for Windows, [OSX installation](http://johnmuschelli.com/neuroc/installing_devtools/index.html) requires XCode and Command Line Tools, GNU/Linux "just works")

# Introduction to the R language

## Base R Data Types: atomic vectors

`numeric` (set seed to sync random number generator):

In [ ]:
set.seed(1)
rnorm(5)

`integer`:

In [ ]:
sample(1L:5L)

`logical`:

In [ ]:
1:3 %in% 3

`character`:

In [ ]:
c("yes", "no")

`factor`:

In [ ]:
factor(c("yes", "no"))

Demo: integer-like properties, `relevel()`

## Base R Data Types: missingness

- Missing Values and others - **IMPORTANT**

In [ ]:
c(NA, NaN, -Inf, Inf)

`class()` to find the class of a variable.

## Base R Data Types: matrix, list, data.frame

`matrix`:

In [ ]:
matrix(1:9, nrow = 3)

The `list` is a non-atomic vector:

In [ ]:
measurements <- c(1.3, 1.6, 3.2, 9.8, 10.2)
parents <- c("Parent1.name", "Parent2.name")
my.list <- list(measurements, parents)
my.list

The `data.frame` has list-like and matrix-like properties:

In [ ]:
x <- 11:16
y <- seq(0, 1, .2)
z <- c("one", "two", "three", "four", "five", "six")
a <- factor(z)
my.df <- data.frame(x, y, z, a, stringsAsFactors = FALSE)

## Bioconductor S4 vectors: DataFrame

- Bioconductor (www.bioconductor.org) defines its own set of vectors using the S4 formal class system
`DataFrame`: like a `data.frame` but more flexible. columns can be any atomic vector type:
    - `GenomicRanges` objects
    - `Rle` (run-length encoding)

In [ ]:
suppressPackageStartupMessages(library(S4Vectors))
df <- DataFrame(var1 = Rle(c("a", "a", "b")),
                var2 = 1:3)
metadata(df) <- list(father = "Levi is my father")
df

## Bioconductor S4 vectors: List and derived classes

In [ ]:
List(my.list)
str(List(my.list))

In [ ]:
suppressPackageStartupMessages(library(IRanges))
IntegerList(var1 = 1:26, var2 = 1:100)
CharacterList(var1 = letters[1:100], var2 = LETTERS[1:26])
LogicalList(var1 = 1:100 %in% 5, var2 = 1:100 %% 2)

## Bioconductor S4 vectors: Biostrings

In [ ]:
suppressPackageStartupMessages(library(Biostrings))
bstring = BString("I am a BString object")
bstring

In [ ]:
dnastring = DNAString("TTGAAA-CTC-N")
dnastring
str(dnastring)

In [ ]:
alphabetFrequency(dnastring, baseOnly=TRUE, as.prob=TRUE)

# dplyr

## Data Manipulation using `dplyr`

* `dplyr` convention aims to ease cognitive burden
* Function names are easy to remember:
1. select (Y)
2. mutate/transmute (add Ys / new Y)
3. filter (get Xs based on condition)
4. slice (get Xs specified)
5. summarise (reduce to single observation) 
6. arrange (re-order observations)

## `dplyr` example

In [ ]:
library(nycflights13)
library(dplyr)
delays <- flights %>%
    filter(!is.na(dep_delay)) %>%
    group_by(year, month, day, hour) %>%
    summarise(delay = mean(dep_delay), n = n()) %>%
    filter(n > 10)
## dplyr cheatsheet
## ggplot2 cheatsheet

## `dplyr` example (cont'd)

In [ ]:
hist(delays$delay, main = "Mean hourly delay", xlab = "Delay (minutes)")

# Lab Exercises

1. [dplyr exercises](http://genomicsclass.github.io/book/pages/dplyr_intro_exercises.html)
2. [random variables exercises](http://genomicsclass.github.io/book/pages/random_variables_exercises.html)

[book]: http://genomicsclass.github.io/book/
[ePub version]: https://leanpub.com/dataanalysisforthelifesciences/
[Source repository]: https://github.com/genomicsclass/labs